In [54]:
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.optimize import linear_sum_assignment

In [58]:
file_path=input('Enter the csv file name: ')

Enter the csv file name:  central_data_phase_2


In [56]:
sheet=input('Enter the sheet name: ')

Enter the sheet name:  na


In [59]:
bus_df = pd.read_csv(f'{file_path}.csv')
bus_df.head()

,Sl No,College/Institute,Zone,Vendor Name,Vehicle Number,Route No,Driver Point Latitude,Driver Point Longitude,Driver Point Name,First Pickup Pt. Latitude,First Pickup Pt. Longitude,First Pickup Point Name,Seat Capacity
0,1,NPSWF,Maqsood,Baghirathi,KA07B1355,1,12.952958,77.804186,Muthkur,12.938821,77.721085,"Balgere, Sobha Dream Acres Apt",33.0
1,2,NPSWF,Maqsood,Baghirathi,KA53D8480,2,12.952708,77.726000,Siddapura,12.900606,77.747839,"Gunjur, Greenwood High School",41.0
2,3,NPSWF,Maqsood,Baghirathi,KA53D8723,3,12.961514,77.615152,Austin Town,12.974034,77.682899,"Vignanagar, Jain Heights East Parade Apt",34.0
3,4,NPSWF,Maqsood,Baghirathi,KA53D8726,4,13.012996,77.698353,K R Puram Extn,12.985157,77.674090,"Kaggadaspura main road, Airtel office",34.0
4,5,NPSWF,Maqsood,Baghirathi,KA53AB9107,5,13.017325,77.657404,Baghirathi Shed,13.013863,77.664028,"Rm nagar, Nellore mess",39.0


In [5]:
office_lat=12.964088807976207
office_lon=77.71808324959153

In [60]:
base_url= "http://localhost:5000/route/v1/driving"

In [61]:
def osrm_calculate(lat1, lon1, lat2, lon2): #input parameters as default lat,lon
    if np.isnan(lat1) or np.isnan(lon1) or np.isnan(lat2) or np.isnan(lon2):
        print("OSRM error: Nan value entered")
        pass
    src=f'{lon1},{lat1};{lon2},{lat2}'
    curl=f"http://localhost:5000/route/v1/driving/{src}?overview=false"
    # time.sleep(0.2)
    try:
        json_response=requests.get(curl).json()
        return json_response['routes'][0]['distance']/1000
    except Exception as exc:
        print(f"OSRM error at {lat1},{lon1} to {lat2},{lon2}: {exc}")
        return np.inf

In [62]:
lat1=input('Latitude 1 column name: ')
lon1=input('Longitude 1 column name: ')
lat2=input('Latitude 2 column name: ')
lon2=input('Longitude 2 column name: ')

Latitude 1 column name:  Driver Point Latitude
Longitude 1 column name:  Driver Point Longitude
Latitude 2 column name:  First Pickup Pt. Latitude
Longitude 2 column name:  First Pickup Pt. Longitude


In [65]:
distance_df=pd.DataFrame(bus_df['Vehicle Number'])
distance_df['dlat']=bus_df[lat1]
distance_df['dlon']=bus_df[lon1]
distance_df['plat']=bus_df[lat2]
distance_df['plon']=bus_df[lon2]
distance_df

,Vehicle Number,dlat,dlon,plat,plon
0,KA07B1355,12.952958,77.804186,12.938821,77.721085
1,KA53D8480,12.952708,77.726000,12.900606,77.747839
2,KA53D8723,12.961514,77.615152,12.974034,77.682899
3,KA53D8726,13.012996,77.698353,12.985157,77.674090
4,KA53AB9107,13.017325,77.657404,13.013863,77.664028
...,...,...,...,...,...
179,KA53AB1810,12.996135,77.757787,13.030863,77.665129
180,KA50A5968,13.017085,77.657374,13.020205,77.656504
181,KA53AB1811,12.985176,77.601569,12.950439,77.642715
182,KA53D5405,13.099021,77.574234,13.064409,77.578175


In [66]:
distance_df['Original dead km'] = bus_df.apply(lambda row: osrm_calculate(row[lat1], row[lon1], row[lat2], row[lon2]), axis=1)

In [67]:
distance_df

,Vehicle Number,dlat,dlon,plat,plon,Original dead km
0,KA07B1355,12.952958,77.804186,12.938821,77.721085,14.3195
1,KA53D8480,12.952708,77.726000,12.900606,77.747839,10.6795
2,KA53D8723,12.961514,77.615152,12.974034,77.682899,10.7253
3,KA53D8726,13.012996,77.698353,12.985157,77.674090,6.9452
4,KA53AB9107,13.017325,77.657404,13.013863,77.664028,1.7180
...,...,...,...,...,...,...
179,KA53AB1810,12.996135,77.757787,13.030863,77.665129,15.9813
180,KA50A5968,13.017085,77.657374,13.020205,77.656504,0.5856
181,KA53AB1811,12.985176,77.601569,12.950439,77.642715,9.1687
182,KA53D5405,13.099021,77.574234,13.064409,77.578175,7.4851


In [8]:
from geopy.geocoders import Nominatim

In [10]:
geolocator = Nominatim(user_agent="distance_calculator")
location = geolocator.reverse("12.95295815,77.80418605")
print(location.address)
# Potsdamer Platz, Mitte, Berlin, 10117, Deutschland, European Union
# >>> print((location.latitude, location.longitude))
# (52.5094982, 13.3765983)
# >>> print(location.raw)
# {'place_id': '654513', 'osm_type': 'node', ...}

Ramaswamypalya, Muthkuru, Hosakote taluk, Bengaluru North, Karnataka, 560067, India


In [51]:
bus_df.head()
distance_df=bus_df

In [36]:
def get_address(lat, lon):
    geolocator = Nominatim(user_agent="distance_calculator")
    location = geolocator.reverse(f'{lat},{lon}')
    print(location)
    return location

In [52]:
distance_df['Driver location'] = distance_df.apply(lambda row: get_address(row['dlat'], row['dlon']), axis=1)
distance_df

Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalore East, Bengaluru Urban, Karnataka, 560036, India
Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalore East, Bengaluru Urban, Karnataka, 560036, India
Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalore East, Bengaluru Urban, Karnataka, 560036, India
Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalore East, Bengaluru Urban, Karnataka, 560036, India
Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalore East, Bengaluru Urban, Karnataka, 560036, India
Amar Jyothi Public School & Pre-University College, Basavanapura Main Road, Shree Vinayaka Layout, Krishnarajapuram, Bangalor

,dlat,dlon,Driver location
0,13.010435,77.701875,(Amar Jyothi Public School & Pre-University Co...
1,13.010127,77.702016,(Amar Jyothi Public School & Pre-University Co...
2,13.010351,77.702046,(Amar Jyothi Public School & Pre-University Co...
3,13.010298,77.702040,(Amar Jyothi Public School & Pre-University Co...
4,13.010177,77.701881,(Amar Jyothi Public School & Pre-University Co...
5,13.010328,77.701658,(Amar Jyothi Public School & Pre-University Co...
6,13.077128,77.731437,"(Manduru, Mandur, Bangalore East, Bengaluru Ur..."
7,13.046138,77.721443,"(Hirandahalli Main Road, Hirandahalli, Bangalo..."
8,13.010256,77.701840,(Amar Jyothi Public School & Pre-University Co...
9,12.981026,77.801004,"(Bodana Hosahalli, Hosakote taluk, Bengaluru N..."


In [42]:
distance_df['Pickup location'] = distance_df.apply(lambda row: get_address(row['plat'], row['plon']), axis=1)
distance_df

2, Panathur Main Road, Balagere, Bangalore East, Bengaluru Urban, Karnataka, 560087, India
SH35, Chikkavaderapura, Heggondahalli, Anekal, Bengaluru Urban, Karnataka, 562125, India
5th Cross Road, Vignananagar, Bangalore East, Bengaluru Urban, Karnataka, 560037, India
Kaggadasapura Main Road, Byrasandra, C V Raman Nagar, Bangalore East, Bengaluru Urban, Karnataka, 560093, India
Vijaya Bank Colony, Banaswadi, Bangalore East, Bengaluru Urban, Karnataka, 560016, India
Ramamurthy Nagara Police Station, Ramamurthinagar Main Road, Dooravaninagar, Ramamurthy Nagar, Bangalore East, Bengaluru Urban, Karnataka, 560016, India
The Empyrean, Baguru Sonnahalli, Maluru taluk, Kolar, Karnataka, 563160, India
Outer Ring Road, Chinnappanahalli, Marathahalli, Bangalore East, Bengaluru Urban, Karnataka, 560037, India
Temple (Ram Temple), Doddanekkundi Main Road, Shamanna Reddy Layout, Bangalore East, Bengaluru Urban, Karnataka, 560037, India
LRDE Layout, Bangalore East, Bengaluru Urban, Karnataka, 560037, 

,Vehicle Number,dlat,dlon,plat,plon,Driver location,Pickup location
0,KA07B1355,12.952958,77.804186,12.938821,77.721085,"(Ramaswamypalya, Muthkuru, Hosakote taluk, Ben...","(2, Panathur Main Road, Balagere, Bangalore Ea..."
1,KA53D8480,12.952708,77.726000,12.900606,77.747839,"(Meenakshi Layout, Bangalore East, Bengaluru U...","(SH35, Chikkavaderapura, Heggondahalli, Anekal..."
2,KA53D8723,12.961514,77.615152,12.974034,77.682899,"(Old Race Course Road, Officers Colony, Austin...","(5th Cross Road, Vignananagar, Bangalore East,..."
3,KA53D8726,13.012996,77.698353,12.985157,77.674090,"(Shree Vinayaka Layout, Krishnarajapuram, Bang...","(Kaggadasapura Main Road, Byrasandra, C V Rama..."
4,KA53AB9107,13.017325,77.657404,13.013863,77.664028,"(Vijaya Bank Colony, Banaswadi, Bangalore East...","(Vijaya Bank Colony, Banaswadi, Bangalore East..."
...,...,...,...,...,...,...,...
120,KA51AE0946,13.046106,77.751673,13.076772,77.734222,"(Konadasanapura, Konadasapura, Bangalore East,...","(Budigere Road, Manduru, Mandur, Bangalore Eas..."
121,KA53AB9558,13.033599,77.720472,13.023457,77.712301,"(Kitthaganur, Medahalli, Bangalore East, Benga...","(Pashmina Waterfront Apartments, TC Palya, See..."
122,KA53AB9560,13.027147,77.776142,13.005213,77.703919,"(Kannamangala, Seegehalli, Bangalore East, Ben...","(Devasandra, Krishnarajapuram, Bangalore East,..."
123,KA03AB2067,13.011984,77.780971,12.997951,77.784225,"(Delhi Public School, Dabaspete - Hosur Highwa...","(Dabaspete - Hosur Highway, Mallasandra, Hosak..."


In [68]:
distance_df.to_csv(f'{file_path}_addresses.csv', index=False)

In [31]:
bus_df['Nodal km'] = bus_df.apply(lambda row: osrm_calculate(row[lat2], row[lon2], office_lat, office_lon), axis=1)

OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808324959153: 'routes'
OSRM error: Nan value entered
OSRM error at nan,nan to 12.964088807976207,77.71808

In [32]:
bus_df.head()

,ID,Name,Mobile No,Email-id,Home Latitude,Home Longitude,Travel km,Nodal Address,Nodal Latitude,Nodal Longitude,Nodal km
0,10118645,Abijith Benny,8.553164e+09,abenny@fossil.com,12.909328,77.664440,10.5987,Haralur Main Rd,12.909375,77.66387,10.5348
1,10122575,ADHEESH C,6.282702e+09,ac@fossil.com,12.909271,77.609100,18.0123,"Kuvempu Nagar, (BTM 2nd Stage, BTM Layout)",12.907893,77.61082,18.0488
2,10108638,Aditi Singh,6.366040e+09,asingh9@fossil.com,12.853173,77.677956,18.4563,NaN,NaN,NaN,inf
3,15002057,Ahamed Ali A,9.880834e+09,ahmed.ali@fossil.com,12.898738,77.639900,14.6536,NaN,NaN,NaN,inf
4,10115985,Aishwarya Mishra,9.821754e+09,amishra01@fossil.com,12.925884,77.729380,6.3703,NaN,NaN,NaN,inf


In [33]:
bus_df.to_excel('Traveller Data.xlsx', index=False)